In [1]:
'''An analysis of how to compute the logarithm of the quantity ξ in Calvetti et. al. , uses the complex valued logarithm. 
As we end up exponentiating the result, Euler's identity results in a number of sign changes, but the results are identical to the lin domain. '''




import sys
sys.path.append('../')
import matplotlib.pyplot as plt
import numpy as np
from utilities.Utils import jacob



rng = np.random.default_rng(5)
theta = np.array([rng.uniform(0.,1.) for _ in range(5)])
weights = np.array([rng.uniform(0.,1.) for _ in range(5)])
weights = weights / np.sum(weights)


'''Computing ln(sum(weights * ln(theta)))'''



expectation_log_theta = 0
for i in range(len(theta)): 
    expectation_log_theta += weights[i] * np.log(theta[i])

print(f"Using the naive approach: {expectation_log_theta}")

deltas = np.zeros(len(theta))

for i in range(len(deltas)): 
    log_theta = np.log(theta[i])
    log_log = np.log(np.abs(log_theta))
    deltas[i] = np.log(weights[i]) + log_log


print(f"ξ using the complex valued log: {-1 * np.exp(jacob(deltas)[-1])}")

ξ = -1 * np.exp(jacob(deltas)[-1])








Using the naive approach: -1.6864274793292386
ξ using the complex valued log: -1.6864274793292384


In [2]:
'''How to compute the logarithm of the quantity Σ in Calvetti et al. using the complex valued logarithm and matrix exponentials.
We have the quantity ξ from above and will resuse it here. '''


psis = np.array(np.log(theta) - ξ)

deltas = np.zeros(len(theta))
for i in range(len(deltas)): 
    deltas[i] = np.log(weights[i]) + np.log(psis[i] ** 2)

print(deltas)

cov = 0
for i in range(len(theta)):
    cov += weights[i] * ((np.log(theta[i]) - ξ) ** 2)
    

print(f"Σ using the naive approach: {cov}")
print(f"Σ using the complex valued log: {np.exp(jacob(deltas)[-1])}")

'''This only covers theta as a single value, we need a slightly more advanced approach for higher dimensional theta vectors'''















[-0.82284349 -0.75446306 -3.68255123 -5.32440985 -0.21486053]
Σ using the naive approach: 1.7461279887162404
Σ using the complex valued log: 1.7461279887162406


'This only covers theta as a single value, we need a slightly more advanced approach for higher dimensional theta vectors'

In [27]:
'''For high dimensional theta, some log terms in the covariance matrix may be negative, requiring us to handle each case differently'''

N = 5
rng = np.random.default_rng(5)
theta = np.array([[rng.uniform(0.,1.) for _ in range(2)] for _ in range(N)])
weights = np.array([rng.uniform(0.,1.) for _ in range(N)])
weights = weights / np.sum(weights)

expectation_log_theta = 0
for i in range(len(theta)): 
    expectation_log_theta += weights[i] * np.log(theta[i])

print(f"ξ using the naive approach: {expectation_log_theta}")



deltas = np.zeros((N,3))


'''Theta matrix needs to be shape num_particles * len(theta)'''
for i in range(theta.shape[1]): 
    sub_theta = theta[:,i]
    for j in range(theta.shape[0]):
        log_theta = np.log(sub_theta[j])
        log_log = np.log(np.abs(log_theta))

        deltas[j,i] = np.log(weights[j]) + log_log


ξ = []
for i in range(theta.shape[1]): 
   ξ.append(-1 * np.exp(jacob(deltas[:,i])))

ξ = np.array(ξ)[:,-1]

print(f"ξ using the complex valued log: {ξ}")

'''Now for Σ '''
psis = np.array(np.log(theta) - ξ)

Σ = 0
for i in range(N):
    Σ += weights[i] * np.outer(psis[i,:] - ξ,psis[i,:]-ξ)

print(f"Σ using the naive approach: \n{Σ}")

'''Using the jacobian logarithm'''

matrix_set = []
for i in range(np.shape(theta)[0]): 
    matrix_set.append(np.outer(psis[i,:] - ξ,psis[i,:]-ξ))
matrix_set = np.array(matrix_set)


for i in range(np.shape(theta)[0]):
    matrix_set[i,:,:] = np.log(np.abs(matrix_set[i,:,:])) + np.full((np.shape(theta)[1],np.shape(theta)[1]),np.log(weights[i]))


Σ = np.zeros((np.shape(theta)[1],np.shape(theta)[1]))
for i in range(np.shape(theta)[1]): 
    for j in range(np.shape(theta)[1]): 
        deltas = matrix_set[:np.shape(theta)[0],i,j]
        Σ[i,j] = np.exp(jacob(deltas)[-1])



print(Σ)



ξ using the naive approach: [-1.61288322 -1.21041598]
ξ using the complex valued log: [-1.61288322 -1.21041598]
Σ using the naive approach: 
[[4.01271731 1.29766223]
 [1.29766223 3.17159   ]]
[[4.01271731 2.25614023]
 [2.25614023 3.17159   ]]
